 # Offboard set of users

In [ ]:
import datetime as dt

from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
from pytz import timezone

from arcgis.gis import GIS

## Log in as organization admin

In [ ]:
gis = GIS(profile="your_gis_admin_profile")

## Get users based on create date/time

In [ ]:
min45 = dt.datetime.now() - dt.timedelta(hours=45)

now_ts_qy = int(dt.datetime.now().timestamp()*1000)
min45_ago_qy = int(min45.timestamp()*1000)

recent_users = [u
               for u in gis.users.search(f"created:[{min45_ago_qy} TO {now_ts_qy}]")]

In [ ]:
recent_users

In [ ]:
for user in recent_users:
    if hasattr(user, 'provisions') and user.provisions:
        print(f"{user.username}\n\t...currently using {len(user.provisions)} license(s):", end="")
        for lic in user.provisions:
            print(f"{' '*3}{lic.title}")  

## Revoke licenses and reassign content before deleting users

In [ ]:
lic_mgr = gis.admin.license

for user in recent_users:
    user.disable()
    try:
        user.delete()
        print(f"...deleted {user.username}")
    except Exception as e:
        if user.items() or [i for f in user.folders 
                            for i in user.items(folder=f)] or gis.groups.search(f"owner:{user.username}"):
            user.reassign_to(gis.users.me.username)
            print(f"...reassigning items and/or groups from {user.username}")
        add_on_provisions = user.provisions
        if add_on_provisions:
            for provision in add_on_provisions:
                provision_license = lic_mgr.get(provision.title)
                provision_license.revoke(user.username, entitlements="*")
        user_bundles = user.bundles
        if user_bundles:
            for bundle in user_bundles:
                bundle.revoke(user)
        user.delete()
        print(f"...deleted {user.username}")
        print(f"\n")